This is an attempt at sketching the generalized equilibrium solver with N-D mapping.

In [3]:
#%%time
def test_func():
    import numpy as np
    from itertools import chain

    x1 = np.random.RandomState(42).rand(10000, 3)
    x1 /= x1.sum(axis=1)[:, np.newaxis]
    x1 = np.concatenate((np.array([[1e-16, 1e-16, 1], [1e-16, 1, 1e-16], [1, 1e-16, 1e-16]]), x1), axis=0)
    energies = 8.3145*1000*np.sum(x1*np.log(x1), axis=-1)
    energies[[0, 1, 2]] = [100000, 100000, 100000]
    dof_values = np.random.RandomState(2).rand(1000, 3)
    dof_values /= dof_values.sum(axis=1)[:, np.newaxis]
    dof_energies = 100000.
    dof_simplices = np.empty(dof_values.shape, dtype=np.int)
    dof_simplices[..., :] = np.arange(x1.shape[-1])
    dof_potentials = np.empty(dof_values.shape, dtype=np.float)
    dof_potentials[...] = np.inf
    trial_points = np.empty(tuple(chain(dof_values.shape[0:-1])), dtype=np.int)
    # Initialize trial point as lowest energy point in the system
    # Trial simplices will be the current simplex with each vertex replaced by the trial point
    trial_points[...] = np.argmin(energies, axis=-1)
    max_iterations = 50
    iterations = 0
    while iterations < max_iterations:
        #print('ITERATION ', iterations)
        trial_simplices = np.empty(tuple(chain(dof_values.shape, [x1.shape[-1]])), dtype=np.int)
        trial_simplices[..., :, :] = dof_simplices[..., np.newaxis, :]
        trial_simplices[..., np.arange(x1.shape[-1]), np.arange(x1.shape[-1])] = trial_points[..., np.newaxis]
        trial_matrix = np.swapaxes(x1[trial_simplices], -1, -2)
        nondegenerate_indices = np.all(np.linalg.svd(trial_matrix, compute_uv=False) > 1e-12, axis=-1)
        nondegenerate_simplices = trial_simplices[nondegenerate_indices]
        dof_sum_array = np.sum(nondegenerate_indices, axis=-1, dtype=np.int)
        dof_index_array = np.repeat(np.arange(dof_values.shape[-2], dtype=np.int), dof_sum_array.astype(np.int))
        fractions = np.linalg.solve(trial_matrix[nondegenerate_indices], dof_values[dof_index_array])
        bounding_simplices = np.all(fractions > 0, axis=-1)
        candidate_simplices = nondegenerate_simplices[bounding_simplices]
        #print('candidate_simplices', candidate_simplices)
        candidate_potentials = np.linalg.solve(x1[candidate_simplices],
                                               energies[candidate_simplices])
        dof_index_array = dof_index_array[bounding_simplices]

        target_values = dof_values[dof_index_array]
        candidate_energies = np.tensordot(candidate_potentials, target_values, axes=(-1, -1))
        #print('candidate energies', len(candidate_energies))
        comparison_matrix = np.where(dof_index_array == np.arange(dof_values.shape[-2])[..., np.newaxis],
                                     candidate_energies, np.inf)
        lowest_candidate_indices = np.argmin(comparison_matrix, axis=-1)
        dof_simplices = np.where((candidate_energies[..., np.arange(dof_values.shape[-2]),
                                                     lowest_candidate_indices] < dof_energies)[..., np.newaxis],
                                 candidate_simplices[lowest_candidate_indices], dof_simplices)
        #print(dof_simplices)
        dof_energies = np.minimum(dof_energies, candidate_energies[..., np.arange(dof_values.shape[-2]),
                                                                   lowest_candidate_indices])
        #print('energies', dof_energies)
        dof_potentials = np.linalg.solve(x1[dof_simplices], energies[dof_simplices])
        #print('simplices', dof_simplices)
        #print('potentials', dof_potentials)
        driving_forces = np.tensordot(dof_potentials, x1, axes=(-1, -1)) - energies
        trial_points = np.argmax(driving_forces, axis=-1)
        #print('trial points', trial_points)
        #print('REMAINING DRIVING FORCE ', np.amax(driving_forces, axis=-1))
        if np.all(driving_forces < 1e-4 * 8.3145 * 1000):
            break
        iterations += 1
    #print(driving_forces)
%lprun -f test_func test_func()
#test_func()

In [8]:
import numpy as np
x1 = np.random.rand(100, 10)
x2 = np.random.rand(100000, 10)
%time x4 = np.tensordot(x1, x2, axes=(-1, -1))
%time x5 = np.inner(x1, x2)
np.allclose(x4, x5)

CPU times: user 393 ms, sys: 25 ms, total: 418 ms
Wall time: 417 ms
CPU times: user 415 ms, sys: 20 ms, total: 435 ms
Wall time: 434 ms


True